In [ ]:
import torch

torch.cuda.empty_cache()

# Enable expandable segments (run before initializing tensors)
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Monitor memory usage
print(torch.cuda.memory_summary())

In [ ]:
from pathlib import Path

model_name = 'yolo11l.pt' # The starting model weights
num_epochs = 5
image_size = 960
batch_size = 4
dataset = Path('football-players-data')
data_yaml_path = dataset / 'data.yaml'
data_yaml_path.exists()
run_name = 'Test'

In [ ]:
from ultralytics import YOLO

model = YOLO(model_name)

print(f"Starting training with model: {model_name}")
print(f"Dataset config: {data_yaml_path}")
print(f"Epochs: {num_epochs}, Image Size: {image_size}, Batch Size: {batch_size}")
print(f"Run Name: {run_name}")

results = model.train(
    data=data_yaml_path,
    epochs=num_epochs,
    imgsz=image_size,
    batch=batch_size,
    name=run_name
)

In [ ]:
import shutil
# Path where YOLO saves the weights
trained_model_path = Path(f"../runs/detect/{run_name}/weights/best.pt")
output_model_dir = Path(f"../models/player_detection/{run_name}")
# Create your custom models directory if it doesn't exist
os.makedirs(output_model_dir, exist_ok=True)

# Move the trained model
shutil.copy(trained_model_path, f"{output_model_dir}/best.pt")
print(f"Model saved to {output_model_dir}/best.pt")